In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [9]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf().setAppName("lab01").setMaster("yarn")
sc = SparkContext(conf=conf)

In [10]:
sc

<SparkContext master=yarn appName=lab01>

In [15]:
rdd = sc.textFile('hdfs:////labs/laba01/ml-100k/u.data') \
        .map(lambda x: x.split("\t"))

In [16]:
rdd.take(1)

[['196', '242', '3', '881250949']]

In [26]:
rddAll = rdd.map(lambda x: (int(x[2]), 1)).reduceByKey(lambda a,b: a+b).sortByKey()

In [27]:
rdd96 = rdd.filter(lambda x: int(x[1]) == 96).map(lambda x: (int(x[2]), 1)).reduceByKey(lambda a,b: a+b).sortByKey()

In [28]:
hist_film = rdd96.collect()

In [29]:
hist_all = rddAll.collect()

In [30]:
hist_film

[(1, 6), (2, 20), (3, 43), (4, 123), (5, 103)]

In [31]:
hist_all

[(1, 6110), (2, 11370), (3, 27145), (4, 34174), (5, 21201)]

In [32]:
lab01 = dict()

In [33]:
lab01["hist_film"] = [x[1] for x in hist_film]
lab01["hist_all"] = [x[1] for x in hist_all]

In [34]:
lab01

{'hist_film': [6, 20, 43, 123, 103],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [35]:
with open('lab01.json', 'w') as f:
    json.dump(lab01, f)

In [41]:
!hdfs dfs -put /data/home/nikolay.abramov/lab01.json /user/nikolay.abramov/lab01.json

In [40]:
!pwd

/data/home/nikolay.abramov
